# Homework 1 #

In this problem you will explore some consequences of the ocean's nonlinear equation of state. Then you will make some calculations regarding air-sea fluxes. Each question is worth 25%. There is an _optional_ bonus question at the end which is worth 10 points towards any future homework.

You will need to calculate thermodynamic quantities from [TEOS-10](http://www.teos-10.org/software.htm). For this you will need to use the TEOS-10 Gibbs Sea Water software. There are versions of this available for your preferred programming environment:

* [TEOS-10 MATLAB](http://www.teos-10.org/software.htm)
* [TEOS-10 GSW for Python](https://github.com/TEOS-10/GSW-Python) (Can be installed with `conda install -c conda-forge gsw`)
* [TEOS-10 GSW for R](https://github.com/TEOS-10/GSW-R)

If you choose to use python for these calculations, then you can [download this assignment](https://github.com/rabernat/intro_to_physical_oceanography/blob/master/homework/01_thermodynamics.ipynb) from github and just fill in the blanks. If you _don't_ go this route, you should still use a computer to type your responses, carefully identifying each question.


## 1) Cabbeling ##

You make two measurements of seawater with a CTD.

| |T (in-situ temperature, $^\circ$C) |S$_p$ (Practical Salinity, PSU)|
|--|--------|-----------------------|
|Measurement 1| 0.0 | 31.0  |
|Measurement 2 | 16.45 | 34.0 |

The measurements are taken at the surface ($p=0$ dbar) in the North Atlantic Ocean, coordinates 45N latitude, 30E longitude.

First convert the measurments to absolute salinity and conservative temperature.

In [7]:
import numpy as np
import gsw

SP1 = 31.0                               # pratical salinity, measurement 1
SP2 = 34.0                               # pratical salinity, measurement 2
p = 0                                    # surface pressure, in dbar
lon = 30                                 # 30E longitude
lat = 45                                 # 45N latitude

SA1 = gsw.SA_from_SP(SP1, p, lon, lat)   # calculate absolute salinities
SA2 = gsw.SA_from_SP(SP2, p, lon, lat)

t1 = 0.0                                 # in-situ temp, measurement 1
t2 = 16.45                               # in-situ temp, measurement 2

CT1 = gsw.CT_from_t(SA1, t1, p)
CT2 = gsw.CT_from_t(SA2, t2, p)

print('Measurement 1:')
print(f'pratical salinity = {SP1:.3f}, absolute salinity = {SA1:.3f}')
print(f'in-situ temp = {t1:.3f}, conservative temp = {CT1:.3f}')
print('Measurement 2:')
print(f'pratical salinity = {SP2:.3f}, absolute salinity = {SA2:.3f}')
print(f'in-situ temp = {t2:.3f}, conservative temp = {CT2:.3f}')


Measurement 1:
pratical salinity = 31.000, absolute salinity = 31.146
in-situ temp = 0.000, conservative temp = 0.016
Measurement 2:
pratical salinity = 34.000, absolute salinity = 34.160
in-situ temp = 16.450, conservative temp = 16.467


>**ANSWER:** Converted measurements (absolute salinity, and convservative temperature) to 3 decimal places:
>
>| |$\Theta$ (conservative temperature, $^\circ$C) |S$_A$ (Absolute Salinity, $g kg^{-1}$)|
>|--|--------|-----------------------|
>|Measurement 1| 0.016 | 31.146 |
>|Measurement 2 | 16.467 | 34.160 |

Now calculate the density of each water parcel? Which water mass is denser? What is their average density?

In [8]:
rho1 = gsw.rho(SA1, CT1, p)                   # calculates in-situ density
rho2 = gsw.rho(SA2, CT2, p)

print(f'Density of measurement 1 = {rho1} kg/m^3')
print(f'Density of measurement 2 = {rho2} kg/m^3')

average_rho = np.mean([rho1, rho2])           # calculates average density
print(f'Average density =  {average_rho:.3f} kg/m^3 (3 decimal places)')

Density of measurement 1 = 1024.8810319849738 kg/m^3
Density of measurement 2 = 1024.87959668733 kg/m^3
Average density =  1024.880 kg/m^3 (3 decimal places)


>**ANSWER:** Parcel 1 (measurement 1) is more dense than parcel 2.

>The average density is $1024.880 \, kg m^{-3}$ (3 d.p.)

>*(The densities are roughly the same - parcels are on an isopycnal)*

Now allow the two water masses to mix. When they mix, they homogenize their conservative temperature and absolute salinity. What is the density of the new water mass?

In [40]:
#need to homogenize CT and SA between measurements, then take density gsw.rho(CT_homo,SA_homo,p)

SA1 = gsw.SA_from_SP(SP1, p1, lon, lat)          # calculates absolute salinity
SA2 = gsw.SA_from_SP(SP2, p2, lon, lat)
CT1 = gsw.CT_from_t(SA1, t1, p1)
CT2 = gsw.CT_from_t(SA2, t2, p2)
mixed_SA = np.mean([SA1, SA2])                   # calculates mixed values of SA and CT - averaged with numpy
mixed_CT = np.mean([CT1, CT2])

mixed_rho = gsw.rho(mixed_SA, mixed_CT, p)       # calculates the density of the new water mass
print(f'Density of new water mass = {mixed_rho:.3f} kg/m^3 (3 d.p.)')

Density of new water mass = 1026.631 kg/m^3 (3 d.p.)


Discuss your result. How does the density of the mixed seawater compare to that of the two source water masses?

>**ANSWER:** 

>The average density is $1026.631 \, kg m^{-3}$ (3 d.p.)

>The density of the mixed seawater is greater than that of the two source water masses. The source water masses had roughly the same densities, however, the new mixed seawater is denser and sinks below both source water masses (downwells). This is cabbeling.

>The density increases because the mixed water parcel has a smaller volume - a net contraction.

>*(The first water mass is colder and fresher, whereas the second water mass is warmer and saltier.)*

---

## 2) Stratification and Thermobaricity ##

You are on cruise in the Antarctic (65S latitude, 20E longitude). You make two measurements:

| |T (in-situ temperature, $^\circ$C) |S$_p$ (Practical Salinity, PSU)| Pressure (dbar) | 
|--|--------|-----------------------|
|Measurement 1| -1.8 | 33.0  | 0 | 
|Measurement 2 | 0.0 | 33.2 | 20 |

Assess the stability of the water column by comparing the densities of the two water masses referenced to the same pressue (i.e. use potential density). Is the water column stably stratified in this region?

In [12]:
SP1 = 33.0                               # pratical salinity, measurement 1
SP2 = 33.2                               # pratical salinity, measurement 2
p1 = 0                                   # pressure in dbar
p2 = 20                                  
lon = 20                                 # 20E longitude
lat = -65                                # 65S latitude

SA1 = gsw.SA_from_SP(SP1, p1, lon, lat)  # calculate absolute salinities
SA2 = gsw.SA_from_SP(SP2, p2, lon, lat)

t1 = -1.8                                # in-situ temp, measurement 1
t2 = 0.0                                 # in-situ temp, measurement 2

CT1 = gsw.CT_from_t(SA1, t1, p1)
CT2 = gsw.CT_from_t(SA2, t2, p2)

# Reference pressures: sigma0 = 0 dbar, sigma1 = 1000 dbar, sigma2 = 2000 dbar, sigma3 = 3000 dbar, sigma4 = 4000 dbar

sig0_1 = gsw.sigma0(SA1, CT1)                  # calculates potential density anomalies with reference pressure 0 dbar.
sig0_2 = gsw.sigma0(SA2, CT1)

rho_1 = gsw.rho_t_exact(SA1, t1, 1000)         # calculates exact densities
rho_2 = gsw.rho_t_exact(SA2, t2, 1000)

average_p = np.mean([p1, p2])
g = gsw.grav(lat,average_p)                    # calculates gravitational acceleration

# gsw.Nsquared
# average_SA = np.mean([SA1, SA2])             # calculates Brunt Vaisala frequency. gsw.Nsqaured was not working for me :( 
# average_CT = np.mean([CT1, CT2])
# average_p = np.mean([p1, p2])
# rho, alpha, beta = gsw.rho_alpha_beta(average_SA, average_CT, average_p)
# Nsquared1 = (g**2)*rho*( beta*(SA1-SA2) - alpha*(CT1-CT2) )*(1/(p1-p2))
# print(Nsquared1)

Nsquared = (g**2)*(sig0_1-sig0_2)*(1/(p1-p2))  # calculates N^2 with potential densities at ref. pressure 0 dbar
N = Nsquared**0.5

print(f'Buoyancy frequency = {Nsquared:.3f} s^(-2), which is positive.')
print(f'The water column stratification is stable in this region. The parcel oscillates vertically, at a frequency N = {N:.3f} s^(-1)')


print(f'Potential density anomaly of parcel 1 = {sig0_1:.3f} kg/m^3')
print(f'Potential density anomaly of parcel 2 = {sig0_2:.3f} kg/m^3')
print('Also, parcel 2 has a greater potential density than parcel 1. Therefore, the stratification is stable, as parcel 2 is below parcel 1')

Buoyancy frequency = 0.784 s^(-2), which is positive.
The water column stratification is stable in this region. The parcel oscillates vertically, at a frequency N = 0.885 s^(-1)
Potential density anomaly of parcel 1 = 26.561 kg/m^3
Potential density anomaly of parcel 2 = 26.724 kg/m^3
Also, parcel 2 has a greater potential density than parcel 1. Therefore, the stratification is stable, as parcel 2 is below parcel 1


Now imagine that ocean circulation transports the same two water masses to pressures of 4990 dbar and 5010 dbar respectively. (One is still approx 20 m deeper than the other.) Compare the two potential densities using the mid-point reference pressure of 5000 dbar. How does the stratification differ?

In [13]:
p_ref = 5000                                                    # dbar
p1 = 4990
p2 = 5010

potential_rho1 = gsw.pot_rho_t_exact(SA1, t1, p1, p_ref)        # calculates potential density (1000kgm^-3 not subtracted)
potential_rho2 = gsw.pot_rho_t_exact(SA2, t2, p2, p_ref)

Nsquared = (g**2)*(potential_rho1-potential_rho2)*(1/(p1-p2))   # calculates N^2 with potential densities at ref. pressure 0 dbar

print(f'Buoyancy frequency = {Nsquared:.3f} s^(-2), which is negative.')
print('The stratification is unstable.')

print(f'Potential density of parcel 1 = {potential_rho1:.3f} kg/m^3')
print(f'Potential density of parcel 2 = {potential_rho2:.3f} kg/m^3')
print('Here, parcel 1 is more dense than parcel 2 - the stratification is unstable, as parcel is above parcel 2.')

Buoyancy frequency = -0.754 s^(-2), which is negative.
The stratification is unstable.
Potential density of parcel 1 = 1049.460 kg/m^3
Potential density of parcel 2 = 1049.304 kg/m^3
Here, parcel 1 is more dense than parcel 2 - the stratification is unstable, as parcel is above parcel 2.


----

## 3) Sensible heat flux

_(NOTE: the following problems mostly do not involve ``gsw``. You can still use a computer as a calculator if you wish.)_

Let's assume that initially the ocean and atmosphere are in a balanced state, with no turbulent heat exchange and $T = T_{10} = 20^\circ$C. Suddenly a cold front comes through and the air temperature drops suddenly to 18$^\circ$C. Assume there is no ocean current and that the atmospheric winds are blowing at 10 m/s. You can also assume that $C_H = 10^{-3}$.


1) Calculate the sensible heat flux immediately after this drop in air temperature.

In [14]:
T_10 = 18              # cold front
T = 20           
C_H = 10**(-3)         # turbulent transfer coefficient
rho_air = 1.3          # kg m^-3
cp_air = 1030          # J kg^-1 K^-1
deltau = 10            # assume no ocean current, and atmospheric winds are at 10m/s
deltaTemp = T_10 - T

sensibleHeatFlux = rho_air*cp_air*C_H*deltau*deltaTemp
print(f'Sensible Heat Flux, Qs = {sensibleHeatFlux} W/m^2')

Sensible Heat Flux, Qs = -26.78 W/m^2


2) Assume the mixed layer is 50 m deep. Calculate the instantaneous rate of mixed layer cooling immediately after the drop in air temperature. Express your answer in degrees per day.

In [41]:
rho_sw = 1025
cp_sw = 3850
h = 50                                                 # mixed layer depth

coolingRate = ( sensibleHeatFlux / (cp_sw*rho_sw*h) )  # in degrees/second
coolingRate = (coolingRate)*24*3600                    # convert to degrees/day
print(f'Rate = {coolingRate:.5f} degrees per day ')

Rate = -0.01173 degrees per day 


3) This cooling rate cannot be maintained. As the mixed layer cools and the ocean temperature approaches the air temperature, the sensible heat flux will become weaker. Derive an equation describing the time evolution of the air-sea temperature difference $T' = T - T_{10}$, assuming $T_{10}$ remains fixed. ($T'$ is positive when the water is warmer than the air.) You should find an equation of the form
$$ \frac{dT'}{dt} = - \lambda T' $$
The constant $\lambda$ is an inverse timescale. Express $\lambda$ first in terms of the other parameters and then find its value. Give your answer in the units days$^{-1}$. In your own words, what does $\lambda$ represent? How does it depend on the mixed layer depth? The wind speed?

>**ANSWER:** The first law of thermodynamics can be written as: $$ \frac{d \mathcal{H}}{dt} = Q $$
>
>where $Q$ is the __air-to-sea heat flux__.
>
>Substitute in the heat content: $$\mathcal{H} = h \rho_{0} c_p^{0} \Theta $$ and sensible heat flux:
>$$
Q_s = \rho^{air} c_p^{air} C_H | \Delta \mathbf{u} | (T_{10} - T) \ .
$$
> Assume for simplicity $T = \Theta$
> to get: $$ \frac{dT'}{dt} = - \frac{\rho^{air} c_p^{air} C_H | \Delta \mathbf{u} |}{\rho_{0} c_p^{0} \, h} \, T' $$
> where, $$ \lambda = \frac{\rho^{air} c_p^{air} C_H | \Delta \mathbf{u} |}{\rho_{0} c_p^{0} \, h} $$

In [42]:
const_lambda = (rho_air*C_H*cp_air*deltau) / (rho_sw*h*cp_sw)  # /sec
const_lambda = const_lambda*24*3600                            # /day

print(f'lambda = {const_lambda:.5f} days^(-1)')

lambda = 0.00586 days^(-1)


>**ANSWER:** lambda = 0.00586 days$^{-1}$.

>lambda represents the rate of change of temperature over the difference in air-sea temperature. The smaller the mixed-layer depth, *i.e.* the more shallow, the more the temperature responds (the greater the rate of change of the air-sea temperature difference). The greater the wind speed, the more the temperature responds (the greater the rate of change of the air-sea temperature difference).

>A small lambda means that the rate of change of air-sea temperature difference is small, i.e. the mixed layer cools slowly.

4) This equation is a simple linear, first-order ordinary differential equation. Write the solution for $T'(t)$. How long does it take the initial air-sea difference of 2$^\circ$C to be reduced by half? How does this compare with the initial cooling rate you calculated in part (b)?

>**ANSWER:**
>$$ \int_{T'_{2}}^{T'_{1}} \frac{dT'}{T'} = - \int_{t_{2}}^{t_{1}} \lambda \: dt $$

>$$ ln\bigg(\frac{T'_{2}}{T'_{1}}\bigg) = - \lambda \left[ t_{1} - t_{2} \right] = - \lambda \, \Delta t$$

>$$ \therefore \;\;\; T'_{2} = T'_{1} \exp(-\lambda \, \Delta t)$$

>$$ \Rightarrow \int_{1^\circ C}^{2^\circ C} \frac{dT'}{T'} = - \int_{t_{2}}^{t_{1}} \lambda \: dt $$

>$$ ln(2) = \lambda \Delta t $$

>$$ \therefore \; \Delta t = \frac{ln(2)}{\lambda} $$


In [44]:
time_difference = np.log(2) / (const_lambda)            # delta time in days
print(f'Time taken = {time_difference:.3f} days')

degreesPerDay = 1/(time_difference)                  
print(f'Rate = -{degreesPerDay:.3f} degrees per day, which is greater than the rate in part(b) by {degreesPerDay/const_lambda:.3f}. But they are of the same order of magnitude.')


Time taken = 118.219 days
Rate = -0.008 degrees per day, which is greater than the rate in part(b) by 1.443. But they are of the same order of magnitude.


----

## 4) Evaporation and Latent Heat Flux

We will now calculate the evaporation and latent heat flux due to the same event. Cold air can hold much less water vapor than warm air. The Clausius-Clapeyron equation describes the maximum amount of water vapor that air can hold. The humidity of air is often expressed as _relative humidity_, i.e. $q$ as a percentage of $q_{sat}(T)$.


1) What is $q_{sat}$, the specific humidity of the right at the sea surface? (The water temperature is 20$^\circ$C, and the air right at the surface is always 100% saturated.)

In [45]:
T = 20 + 273
q_sat_surface = 0.98*6.4e5*(1/rho_air)*np.exp(-5107.4/T)
print(f'q_sat = {q_sat_surface:.4f}')

q_sat = 0.0130


2) Assume that the air in the cold front has 50% relative humidity at 10 m above the sea surface. What is $q_{10}$? (Recall that $T_{10} = 18^\circ$C.)

In [46]:
T = 18 + 273
RH = 0.5                                             # relative humidity
q_sat_10 = 0.98*6.4e5*(1/rho_air)*np.exp(-5107.4/T)
q_10 = q_sat_10*RH
print(f'q_10 = {q_10:.4f}')

q_10 = 0.0058


3) Calculate the evaporation rate immediately after the drop in air temperature. What are the units of $E$? Rainfall is often measured in units of mm per day. Can you convert your answer to these units?

In [47]:
C_E = 1e-3
E = -(rho_air)*(C_E)*(deltau)*(q_10 - q_sat_10)      # evaporation rate equation
print(f'Evaporation rate = {E} kg m^-2 s^-1')
E_mm_day = ( E / rho_fw )*(1000)*(24*3600)          # convert to units of mm per day, by diving by density of seawater..
print(f'Evaporation rate = {E_mm_day} mm per day')

Evaporation rate = 7.481446063504483e-05 kg m^-2 s^-1
Evaporation rate = 6.4639693988678735 mm per day


4) Calculate the instantaneous rate of change of the mixed layer salinity due to evaporation immediately after the drop in air temperature using the virtual salt flux approximation. Assume that the average salinity of the mixed layer $S_{A0} = 35$ g/kg.

In [48]:
SA_0 = 35                        # average salinity of the mixed layer (g/kg)
rho_fw = 1000                    # density of freshwater
dS_dt = (SA_0*E)/(h*rho_fw)      # virtual salt flux approx.
print(f'Instantaneous rate of change of mixed layer salinity due to evaporation =  {dS_dt} g/kg s^(-1)')    #UNITS g/(kg s)

Instantaneous rate of change of mixed layer salinity due to evaporation =  5.2370122444531385e-08 g/kg s^(-1)


5) Calculate the latent heat flux corresponding with this evaporation rate.

In [49]:
L_e = 2.5e6                      # Latent heat of vaporisation (J/kg)
latent_heat_flux = -L_e*E        # calculates latent heat flux (W/m^2)
print(f'Latent Heat Flux = {latent_heat_flux:.3f} W/m^2')

Latent Heat Flux = -187.036 W/m^2


6) Sensible heat fluxes can be both positive and negative over the ocean, depending on whether the air temperature is warmer or cooler than the water. As a result, large cancellations occur, and sensible heat fluxes do not contribute very much to the global heat budget. In contrast, latent heat loss is only negative, and strongly cools the ocean as a whole. Why is latent heat flux always negative?

>**ANSWER:** Latent heat flux is the flux of heat from the ocean surfaces, via evaporation. Seawater evaporates, cooling the ocean, and condenses to atmospheric water vapour, releasing the latent heat of vaporisation into the atmosphere.
>
The process requires energy, and extracts it via heat from the ocean, thus cooling the ocean. Therefore, latent heat flux is always negative.

---

## 5) OPTIONAL Bonus Question: Total Density Flux

All of these processes (sensible heat loss, evaporation, and latent heat loss) are making the mixed layer denser. Let's compare their effect

1. Combine the equations for the rate of change of $T$ and $S_A$ together to form an expression for the rate of change of density $\rho$. (You may assume for simplicity that $T=\Theta$.) You will need to use the equation of state. (Since the pressure changes over the mixed layer are small, you should neglect the dependence of density on pressure.)
1. Quantify the densification rate of the mixed layer due to each process immediately following the change in air temperature. Use ``gsw`` to calculate the appropriate thermal expansion and haline contraction coefficients. Express your answer in kg m$^{-3}$ per day. Which is the dominant process?
1. The mixed layer is the lightest (least dense) water in the water column. It overlies the stratified pycnocline. What happens when air-sea fluxes cause the mixed layer to become denser than the water below it?


$$ \frac{d \rho}{dt} \simeq \rho_0 \left [ \alpha_0 \lambda (T - T_{10}) -  \gamma_C (T - T_{10}) \frac{dT}{dt} + \beta_0 \frac{dS}{dt} \right ]
$$


In [50]:
T = 18 + 273
rateChange_density = ((SA_0*E)/ (rho_fw))*(1/dS_dt)*sensibleHeatFlux*(1/T)*(1/cp_sw)

In [51]:
alpha = gsw.alpha(SA_0, T, 0)         # calculates thermal expansion coeff
beta = gsw.beta(SA_0, T, 0)           # calculates haline contraction coeff
cabbeling = gsw.cabbeling(SA_0, T, 0)
dT_dt = sensibleHeatFlux*(1/rho_sw)*(1/h)*(1/cp_sw)

rateChange_density = rho_sw*(alpha*const_lambda*(293-T) - cabbeling*(293-T)*dT_dt + beta*(dS_dt))
print(f'Rate of change of density = {rateChange_density:.5f} kg m^-3 per second')
rateChange_density_day = rateChange_density*24*3600
print(f'Rate of change of density = {rateChange_density_day:.5f} kg m^-3 per day')
print('Cabbeling is the dominant process.')

Rate of change of density = 0.26714 kg m^-3 per second
Rate of change of density = 23080.78772 kg m^-3 per day
Cabbeling is the dominant process.


>**ANSWER:** When the air-sea fluxes cause the mixed layer to become denser than the water below it, it sinks (downwells). The dense surface water sinks into the deep ocean. E.g., the North Atlantic Deep Water is formed this way. The deep Atlantic water is saltier than the rest of the water column above it.